In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

## import matplotlib, etc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.18.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://markdatabucket.s3.amazonaws.com/cbb.csv"

spark.sparkContext.addFile(url)
spark_data_df = spark.read.csv(SparkFiles.get("cbb.csv"), sep=",", header=True, inferSchema=True)

# need to convert spark dataframe to pandas dataframe
cbb_df = spark_data_df.toPandas()

# Show DataFrame
cbb_df.head()

TEAM CONF   G   W  ADJOE  ...  ADJ_T   WAB  POSTSEASON  SEED  YEAR
0  North Carolina  ACC  40  33  123.3  ...   71.7   8.6         2ND     1  2016
1       Wisconsin  B10  40  36  129.1  ...   59.3  11.3         2ND     1  2015
2        Michigan  B10  40  33  114.4  ...   65.9   6.9         2ND     3  2018
3      Texas Tech  B12  38  31  115.2  ...   67.5   7.0         2ND     3  2019
4         Gonzaga  WCC  39  37  117.8  ...   71.5   7.7         2ND     1  2017

[5 rows x 24 columns]

In [5]:
# duplicate postseason column to later use as binary option
cbb_df['POSTSEASON_BINARY'] = cbb_df['POSTSEASON']
cbb_df.head()

TEAM CONF   G   W  ...  POSTSEASON  SEED  YEAR  POSTSEASON_BINARY
0  North Carolina  ACC  40  33  ...         2ND     1  2016                2ND
1       Wisconsin  B10  40  36  ...         2ND     1  2015                2ND
2        Michigan  B10  40  33  ...         2ND     3  2018                2ND
3      Texas Tech  B12  38  31  ...         2ND     3  2019                2ND
4         Gonzaga  WCC  39  37  ...         2ND     1  2017                2ND

[5 rows x 25 columns]

In [6]:
cbb_df['POSTSEASON_BINARY'] = cbb_df['POSTSEASON']
cbb_df.POSTSEASON_BINARY[cbb_df.POSTSEASON_BINARY == "NA"] = 0
cbb_df.POSTSEASON_BINARY[cbb_df.POSTSEASON_BINARY != 0] = 1
cbb_df.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


TEAM CONF   G   W  ...  POSTSEASON  SEED  YEAR  POSTSEASON_BINARY
0  North Carolina  ACC  40  33  ...         2ND     1  2016                  1
1       Wisconsin  B10  40  36  ...         2ND     1  2015                  1
2        Michigan  B10  40  33  ...         2ND     3  2018                  1
3      Texas Tech  B12  38  31  ...         2ND     3  2019                  1
4         Gonzaga  WCC  39  37  ...         2ND     1  2017                  1
5            Duke  ACC  39  35  ...   Champions     1  2015                  1
6        Virginia  ACC  38  35  ...   Champions     1  2019                  1
7  North Carolina  ACC  39  33  ...   Champions     1  2017                  1
8       Villanova   BE  40  35  ...   Champions     2  2016                  1
9       Villanova   BE  40  36  ...   Champions     1  2018                  1

[10 rows x 25 columns]

In [7]:
# Assign the data to X and y
# Note: Sklearn requires a two-dimensional array of values
# so we use reshape to create this

X = cbb_df['3P_D'].values.reshape(-1, 1)
y = cbb_df.POSTSEASON_BINARY
y=y.astype('int')

In [8]:
# Plot the data
plt.scatter(X, y)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [10]:
# Create the model and fit the model to the data

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:

classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8003037205770691
Testing Data Score: 0.8090909090909091


In [13]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [0 0 0 0 0 0 0 0 0 0]
First 10 Actual labels: [0, 1, 1, 0, 0, 0, 0, 0, 0, 0]


In [14]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

Prediction  Actual
0             0       0
1             0       1
2             0       1
3             0       0
4             0       0
..          ...     ...
435           0       1
436           0       0
437           0       0
438           1       1
439           0       1

[440 rows x 2 columns]